# 📝 Interactive To-Do List (Notebook Version)
This notebook implements a persistent Task Manager.
* **Data Storage:** Tasks are saved to `tasks.json`.
* **Interface:** Uses a simple text-based menu.
* **Compatibility:** This version is designed to run within a Jupyter Notebook using `clear_output` to refresh the display.

In [7]:
import json
import os
import time
from IPython.display import clear_output

TODO_FILE = "tasks.json"

def clear_screen():
    """Clears the terminal screen"""
    os.system('clear')

### 1. Persistence Layer (Load & Save)
These functions handle the file I/O operations.
* **Load:** safely reads the JSON file, returning an empty list if the file is missing or corrupted.
* **Save:** Writes the list back to the disk immediately after any change.

In [8]:
def load_tasks():
    """Loads tasks from the JSON file."""
    if not os.path.exists(TODO_FILE):
        return []
    try:
        with open(TODO_FILE, 'r') as file:
            return json.load(file)
    except (json.JSONDecodeError, IOError):
        return []

def save_tasks(tasks):
    """Saves the current list of tasks to the JSON file."""
    try:
        with open(TODO_FILE, 'w') as file:
            json.dump(tasks, file, indent=4)
    except IOError as e:
        print(f"Error saving file: {e}")
        time.sleep(2)

### 2. Core Actions (CRUD)
These functions modify the task list:
* **Add:** Appends a new task dictionary.
* **Complete:** Updates the status flag of a specific task.
* **Delete:** Removes a task by index.
* *Note:* All these functions call `save_tasks()` immediately to ensure data is never lost.

In [9]:
def add_task(tasks, title):
    """Adds a new task."""
    tasks.append({"title": title, "completed": False})
    save_tasks(tasks)
    return f"✓ Added: '{title}'"

def complete_task(tasks, task_num):
    """Marks a task as complete."""
    if 1 <= task_num <= len(tasks):
        tasks[task_num - 1]['completed'] = True
        save_tasks(tasks)
        return f"✓ Task {task_num} marked as complete."
    return "⚠ Invalid task number."

def delete_task(tasks, task_num):
    """Deletes a task."""
    if 1 <= task_num <= len(tasks):
        removed = tasks.pop(task_num - 1)
        save_tasks(tasks)
        return f"✗ Deleted: '{removed['title']}'"
    return "⚠ Invalid task number."

### 3. Display Logic
This function formats the list for the user.
* It converts the boolean `True/False` into visual indicators `[x]` or `[ ]`.
* It prints index numbers so the user can easily reference tasks by ID.

In [10]:
def view_tasks(tasks):
    """Displays the list of tasks with index numbers and status."""
    print("\n--- YOUR TO-DO LIST ---")
    
    if not tasks:
        print("No tasks recorded.")
        return

    for index, task in enumerate(tasks):
        status = "[x]" if task['completed'] else "[ ]"
        print(f"{index + 1}. {status} {task['title']}")
        
    print("-----------------------")

### 4. Main Application Loop
The entry point of the program.
* **Screen Clearing:** uses `clear_output(wait=True)` to create a dynamic "app-like" feel inside the notebook (Note: You may need to run `from IPython.display import clear_output` for this to work).
* **Feedback Loop:** captures status messages (like "Task Deleted") and displays them at the top of the next screen refresh so the user knows their action succeeded.

In [11]:
def main():
    tasks = load_tasks()
    last_message = ""
    
    while True:
        # 1. Clear the screen FIRST
        clear_output(wait=True)
        
        # 2. Print the list
        view_tasks(tasks)
        
        # 3. Print the status message (Saved/Deleted etc)
        if last_message:
            print(f"\n{last_message}")
            last_message = ""
            
        # 4. Input blocks the loop here until you type
        print("\nOptions: [1] Add  [2] Refresh  [3] Complete  [4] Delete  [5] Exit")
        choice = input("cmd> ").strip()

        if choice == '1':
            title = input("Description: ").strip().title()
            if title: last_message = add_task(tasks, title)
        
        elif choice == '2':
            pass
            
        elif choice == '3':
            try:
                num = int(input("Task #: "))
                last_message = complete_task(tasks, num)
            except ValueError:
                last_message = "⚠ Invalid number."

        elif choice == '4':
            try:
                num = int(input("Task #: "))
                last_message = delete_task(tasks, num)
            except ValueError:
                last_message = "⚠ Invalid number."

        elif choice == '5':
            print("Bye!")
            break

if __name__ == "__main__":
    main()


--- YOUR TO-DO LIST ---
1. [x] Break Fast
2. [ ] Lunch
-----------------------

Options: [1] Add  [2] Refresh  [3] Complete  [4] Delete  [5] Exit
Bye!
